In [10]:
from PandaStation import *
from PandaGrasping import *

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

# Let's do all of our imports here, too.
import numpy as np
import ipywidgets
import pydot
import pydrake.all
import os
from IPython.display import display, SVG
import subprocess


import pydrake.all
from pydrake.geometry import Cylinder, Box
from pydrake.all import (
    RigidTransform, RotationMatrix, AngleAxis, RollPitchYaw, InverseKinematics, MultibodyPlant, Parser,
    FindResourceOrThrow, Solve, PiecewisePolynomial, TrajectorySource, SceneGraph, DiagramBuilder,
    AddMultibodyPlantSceneGraph, LinearBushingRollPitchYaw, MathematicalProgram, AutoDiffXd, GenerateHtml, Role,
    LeafSystem, AbstractValue, PublishEvent, TriggerType, BasicVector, PiecewiseQuaternionSlerp,
    RandomGenerator, UniformlyRandomRotationMatrix, ConnectMeshcatVisualizer
    )
import pydrake.perception as mut
import open3d as o3d
from ompl import base as ob
from ompl import geometric as og
import time
from enum import Enum

ycb = {"cracker": "drake/manipulation/models/ycb/sdf/003_cracker_box.sdf", 
    "sugar": "drake/manipulation/models/ycb/sdf/004_sugar_box.sdf", 
    "soup": "drake/manipulation/models/ycb/sdf/005_tomato_soup_can.sdf", 
    "mustard": "drake/manipulation/models/ycb/sdf/006_mustard_bottle.sdf", 
    "gelatin": "drake/manipulation/models/ycb/sdf/009_gelatin_box.sdf", 
    "meat": "drake/manipulation/models/ycb/sdf/010_potted_meat_can.sdf",
    "brick": "drake/examples/manipulation_station/models/061_foam_brick.sdf"}

for key in ycb.keys():
    ycb[key] = FindResourceOrThrow(ycb[key])
    
ycb["puck"] = FindResource("models/puck.urdf")
ycb["marble"] = FindResource("models/marble.urdf")
ycb["dumbbell"] = FindResource("models/dumbbell.urdf")

def ycb_resource(name):
    global ycb
    return ycb[name]

def random_ycb_resource():
    global ycb
    ycb_items = list(ycb.items())
    index = np.random.randint(0, len(ycb_items))
    return ycb_resource(ycb_items[index][0])

def random_position():
    return [0.6 + np.random.uniform(-0.05, 0.05), np.random.uniform(-0.1,0.1), 0.2]

def random_orientation():
    rot = RotationMatrix(RollPitchYaw(np.random.uniform(-np.pi, np.pi),
                                     np.random.uniform(-np.pi, np.pi),
                                     np.random.uniform(-np.pi, np.pi)))
    return rot

In [20]:
builder = DiagramBuilder()

station = builder.AddSystem(OmniStation())
station.SetupTableStation(welded_hand = True)

station.AddModelFromFile(ycb_resource("dumbbell"),
                         RigidTransform(random_orientation(), random_position()))

station.Finalize()

station_context = station.CreateDefaultContext()
scene_graph = station.get_scene_graph()
scene_graph_context = station.GetSubsystemContext(scene_graph, station_context)
plant = station.get_multibody_plant()
plant_context = station.GetSubsystemContext(plant, station_context)
omni = station.GetOmni()

v = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url)#, role = Role.kProximity)

t1 = time.time()
ik_station, body_infos = create_welded_station(station, station_context, omni = True)
ik_station_context = ik_station.CreateDefaultContext()

#for shape in body_infos[0].shape_infos:
#    print(shape)
q, cost = grasp_pose(body_infos[0], ik_station, ik_station_context, q_nominal = None)
print(f"q: {q}\ncost {cost}")
print(f"time: {time.time() - t1}")

diagram = builder.Build()
context = diagram.CreateDefaultContext()

#if np.isfinite(cost):
plant_context = plant.GetMyContextFromRoot(context)
plant.SetPositions(plant_context, omni, q)

station_context = station.GetMyContextFromRoot(context)
scene_graph = station.get_scene_graph()
scene_graph_context = station.GetSubsystemContext(scene_graph, station_context)
query_output_port = scene_graph.GetOutputPort("query")
query_object = query_output_port.Eval(scene_graph_context)

v.load()
diagram.Publish(context)

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6034...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7034/static/
Connected to meshcat-server.
q: [0.63255113 0.05078414 0.24420768 2.16259067 0.         2.07428852]
cost inf
time: 0.38672924041748047
